# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

   ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/990.0 kB 660.6 kB/s eta 0:00:02
   -- ------------------------------------ 71.7/990.0 kB 787.7 kB/s eta 0:00:02
   ------ --------------------------------- 153.6/990.0 kB 1.1 MB/s eta 0:00:01
   ------------ --------------------------- 317.4/990.0 kB 1.6 MB/s eta 0:00:01
   -------------------- ------------------- 512.0/990.0 kB 2.1 MB/s eta 0:00:01
   ---------------------------- ----------- 716.8/990.0 kB 2.5 MB/s eta 0:00:01
   ----------------------------------- ---- 890.9/990.0 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------  983.0/990.0 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 990.0/990.0 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/373.5 kB ? eta -:--:--
   --------------------------------------  368.6/373.5 kB 23.9 MB/s eta 0:00:01
   ---------------------------------------- 373.5/373.5 kB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires openai<2.0.0,>=1.24.0, but you have openai 1.14.2 which is incompatible.


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [3]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [5]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1", api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")

In [6]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

d:\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
d:\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [7]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [8]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [10]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [11]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题']


In [12]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题


In [13]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [14]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [14]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.invoke("黑体辐射是什么？?"))

{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指具有完美吸收和辐射能力的理想热辐射体所发出的电磁辐射。它的特点是辐射出的能量和频率分布与温度有关，随着温度的升高，辐射出的能量和频率也会增加。黑体辐射是理解热力学和量子力学的重要概念，对研究物质的热辐射和光谱学有很大的意义。'}


In [16]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要确定大于40的质数。40不是质数，我们从41开始检查。41是质数，但41+1=42不能被3整除。接下来是43，43+1=44也不能被3整除。然后是47，47+1=48可以被3整除。所以大于40的第一个质数是47，47+1=48可以被3整除。'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度和强大引力的天体，它的重力场非常强大，甚至连光也无法逃脱。它的存在是由于恒星在死亡时发生了坍缩，使得它们的质量集中在极小的空间内。黑洞的存在已经被多项观测和理论证据所证实，但仍然有很多未解之谜。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [1]:
biologic_template = """你是一位非常聪明的生物学教授。
你擅长以简洁易懂的方式回答关于生物相关的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


computer_template = """你是一位很棒的计算机学家。你擅长回答计算机问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

chinese_template = """你是一位汉语言文学老师。你擅长回答汉语言问题。
之所以如此出色，是因为你了解中国历史悠久的文化，你会参考历史回答问题。

这是一个问题：
{input}"""

prompt_infos = [
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biologic_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_template,
    },
]

In [4]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1", api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")

In [5]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

d:\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
d:\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [6]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [7]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [8]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [9]:
print(chain.invoke("CPU是什么？?"))



> Entering new MultiPromptChain chain...
计算机: {'input': '什么是CPU?'}
> Finished chain.
{'input': '什么是CPU?', 'text': '\n\nCPU是计算机的中央处理器，它是一个微型芯片，负责执行计算机程序的指令并控制计算机的各种操作。它包含算术逻辑单元（ALU）、控制单元（CU）和寄存器，通过读取和解释存储在内存中的指令，来实现算术运算、逻辑运算、数据传输等功能。它也被称为计算机的大脑。'}


In [10]:
print(chain.invoke("简述生物计算机的可行性"))



> Entering new MultiPromptChain chain...
None: {'input': '简述生物计算机的可行性'}
> Finished chain.
{'input': '简述生物计算机的可行性', 'history': '', 'text': ' 生物计算机是指利用生物体内的化学反应和生物分子来进行计算和信息存储的计算机。它的可行性主要是基于生物体内的化学反应和分子之间的相互作用能够实现类似于传统计算机中的逻辑门和存储单元的功能。这种技术目前仍处于研究阶段，但已经取得了一些进展。例如，一些科学家已经成功地在细菌中构建了简单的逻辑门，并且也有研究团队成功地将信息存储在DNA分子中。因此，可以说生物计算机的可行性是有一定的基础的，但仍需要更多的研究和发展来实现其真正的应用。'}


In [12]:
print(chain.invoke("人为什么活着"))



> Entering new MultiPromptChain chain...
生物: {'input': '人为什么活着'}
> Finished chain.
{'input': '人为什么活着', 'text': '？\n\n回答：\n\n人类之所以活着，是因为我们具有生命的本能和欲望。生物学角度来看，我们的生理机能驱使我们生存下去，包括呼吸、进食、繁殖等。但是作为高等生物，我们也具有更加复杂的心理和情感需求，比如追求幸福、实现梦想、建立人际关系等。总的来说，人类活着的意义是多样的，每个人都有不同的理解和追求。'}
